In [1]:
from pydriller import RepositoryMining

In [2]:
config = {
    "url": "../data/react-vis"
}

In [19]:
class Directory:
    
    ## class Directory maintains a static snapshot of a directory structure
    
    def __init__(self):
        self.sub_directories = {}
        self.files = {}
        self.size = 0
        self.freeze = False
        self.name = '/'
    
    def add_file(self, path, filename, file, size = 1):
        
        if path == '.':
            self.files[filename] = file
        else:
            sub_directory = path.split('/')[0]
            remaining_path = '.'
            if len(path.split('/')) > 1:
                remaining_path = '/'.join(path.split('/')[1:])
            if sub_directory not in self.sub_directories: #create new directory
                self.sub_directories[sub_directory] = Directory()
            self.sub_directories[sub_directory].add_file(remaining_path, filename, file)
        
        self.size += size
    
    def remove_file(self, path, filename, file, size = 1):
        
        if path == '.':
            del self.files[filename]
        else:
            sub_directory = path.split('/')[0]
            remaining_path = '.'
            if len(path.split('/')) > 1:
                remaining_path = '/'.join(path.split('/')[1:])
            self.sub_directories[sub_directory].remove_file(remaining_path, filename, file)
            
        self.size -= size
    
    def rename_file(self, path, old_filename, new_filename, file, size = 1):
        
        self.remove_file(path, old_filename, file, size)
        self.add_file(path, new_filename, file, size)
        
    def modify_file(self, path, filename, size_change = 0):
    
        self.size += size_change
        if path != '.':
            sub_directory = path.split('/')[0]
            remaining_path = '/'.join(path.split('/')[1:])
            self.sub_directories[sub_directory].modify_file(remaining_path, filename, size_change)
        
    def clean_print(self, level=0):
        
        ## print a depth first tranversal of the tree
        print(' ' * level + self.name)
        for directoryname in self.sub_directories:
            self.sub_directories[directoryname].clean_print(level = level + 1)
        for filename in self.files:
            print(' ' * (level + 1) + filename)
        
    def to_json_tree(self):
        
        pass
    
# class Delta: # delta records the directory structure changes
#     def 


In [21]:
root = Directory()

root.add_file('.', 'a', None)
root.add_file('b/c/d', 'e', None)
root.add_file('b/c/d', 'e', None)
root.add_file('b/c/e', 'e', None)


root.clean_print()



/
 /
  /
   /
    e
 a


In [4]:

i, limit = 0, 10
for commit in RepositoryMining(path_to_repo=[config["url"]]).traverse_commits():
    if i < limit:
        print("Project {}, commit {}, author {}, date {}".format(
            commit.project_path, commit.hash, commit.author.email, commit.author_date, commit.msg))
        i += 1
        for modification in commit.modifications:
            print("\t File {}: change type {} added {}, removed {} ".format(modification.new_path, modification.change_type , modification.added, modification.removed))
        
    else:
        break


Project ../data/react-vis, commit 339eb5d29ea76150564d253908a29d1d893adfa7, author antonb@uber.com, date 2016-02-22 13:43:10-08:00
	 File .babelrc: change type ModificationType.ADD added 3, removed 0 
	 File .editorconfig: change type ModificationType.ADD added 11, removed 0 
	 File .eslintrc: change type ModificationType.ADD added 456, removed 0 
	 File .gitignore: change type ModificationType.ADD added 15, removed 0 
	 File .npmignore: change type ModificationType.ADD added 4, removed 0 
	 File LICENSE: change type ModificationType.ADD added 19, removed 0 
	 File README.md: change type ModificationType.ADD added 40, removed 0 
	 File docs/common-principles.md: change type ModificationType.ADD added 50, removed 0 
	 File docs/radial-chart.md: change type ModificationType.ADD added 0, removed 0 
	 File docs/table.md: change type ModificationType.ADD added 45, removed 0 
	 File docs/treemap.md: change type ModificationType.ADD added 85, removed 0 
	 File docs/xy-plot.md: change type Mod